In [1]:
import os
import json
import numpy
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import numpy as np
import pandas as pd
import calendar
import datetime
from IPython.display import Markdown, display, Javascript, display_html

def df_style(header=20, body=16, max_width=100):
    dfstyle = [dict(selector="th", props=[('font-size', f'{header}px')]),
               dict(selector="td", props=[('font-size', f'{body}px')]),
               dict(selector="td", props=[('text-align', 'center')]),
               dict(selector="th", props=[('text-align', 'center')]),
               dict(selector='th', props=[('max-width',f'{max_width}px;')])]
    return dfstyle

In [2]:
# ----- INPUTS HERE ----- #
building_name = 'Testing'
f = open(f'./response_1678286136012.json')
customer_name = 'Testing'

In [3]:
data = json.load(f)
file_name = os.path.splitext(f.name)[0]

<img src=https://res.cloudinary.com/crunchbase-production/image/upload/c_lpad,f_auto,q_auto:eco,dpr_1/htdkfr3co4n7wyxpslr0 alt="Drawing" style="width: 500px;" align="left"/>

In [4]:
display(Markdown(f"# Carbon Reduction Plan for {building_name}"))
display(Markdown(f"## Prepared for {customer_name}"))
display(Markdown(f"### {datetime.datetime.now().strftime('%B %Y')}"))
display(Markdown("***"))

# Carbon Reduction Plan for Wildrose Residence

## Prepared for Testing

### March 2023

***

### Building Summary

In [5]:
building_details_data = [
    data['base_building_state']['building_archetype'],
    data['base_building_state']['gross_floor_area'],
    data['base_building_state']['floors_above_grade']
]

In [6]:
building_details = pd.DataFrame(
    data=building_details_data,
    index=[
        'Building Type',
        'Floor Area (m²)',
        'Number of Floors'
    ],
    columns=['']
)
# building_details
display(building_details.T.style.set_table_styles(df_style(header=16, body=16, max_width=150)))

,Building Type,Floor Area (m²),Number of Floors
,multi_unit_residential,46.880000,1


### Carbon Reduction Plan Inputs

In [7]:
#data from json
crpo_run_options = data['raw_data']['crpo_run_options']

df_options = pd.DataFrame.from_records(crpo_run_options,index=[''])
df_options.columns = df_options.columns.str.replace("_"," ").str.title()

# fka as df1
df_weights = df_options[['Weight Abatement Cost','Weight Emission Saving','Weight Net Present Value','Weight Payback Years']].round(2)

# fka as df2
df_carbon_scenario = df_options.loc[:,['Carbon Scenario','Net Present Value Discount Rate']]
df_carbon_scenario['Carbon Scenario'] = df_carbon_scenario['Carbon Scenario'].str.replace('_'," ").str.title()
df_carbon_scenario.loc["",["2023 Carbon Price per Tonne","2030 Carbon Price per Tonne","2050 Carbon Price per Tonne"]] = [65,170,300]

# fka as df3
elec_gas_rates_data = [
    data['building_analysis']['yearly_summaries'][0]['utility_rates']['electricity'],
    data['building_analysis']['yearly_summaries'][0]['utility_rates']['natural_gas']
]
df_elec_gas_rates = pd.DataFrame(
    data=elec_gas_rates_data,
    index=['Electricity Cost ($/kWh)','Natural Gas Cost ($/kWh)'],
    columns=[""]
).T

### print dataframes side by side
space = "\xa0" * 5

df_weights_s = df_weights.style.set_table_attributes("style='display:inline'")
df_carbon_scenario_s = df_carbon_scenario.style.set_table_attributes("style='display:inline'")
df_elec_gas_rates_s = df_elec_gas_rates.style.set_table_attributes("style='display:inline'")

df_weights_mapper = {k: "{0:.2f}" for k in df_weights.columns}
df_carbon_scenario_mapper = {k: "${0:.0f}" for k in ['2023 Carbon Price per Tonne','2030 Carbon Price per Tonne','2050 Carbon Price per Tonne']}
df_carbon_scenario_mapper['Net Present Value Discount Rate'] = '{:,.0%}'
df_elec_gas_rates_mapper = {k: "{0:.4f}" for k in df_elec_gas_rates.columns}

display_html(df_weights_s.set_table_styles(df_style(header=16, body=16, max_width=75)).format(df_weights_mapper)._repr_html_()
             +space+df_carbon_scenario_s.set_table_styles(df_style(header=16, body=16, max_width=75)).format(df_carbon_scenario_mapper)._repr_html_()
             +space+df_elec_gas_rates_s.set_table_styles(df_style(header=16, body=16, max_width=100)).format(df_elec_gas_rates_mapper)._repr_html_(), raw=True)


Weight Abatement Cost 
 Weight Emission Saving 
 Weight Net Present Value 
 Weight Payback Years 
 
 
 
 
 
 0.00 
 0.65 
 0.35 
 0.00 
 
 
 
     
 
 
 
   
 Carbon Scenario 
 Net Present Value Discount Rate 
 2023 Carbon Price per Tonne 
 2030 Carbon Price per Tonne 
 2050 Carbon Price per Tonne 
 
 
 
 
 
 Current Policies 
 3% 
 $65 
 $170 
 $300 
 
 
 
     
 
 
 
   
 Electricity Cost ($/kWh) 
 Natural Gas Cost ($/kWh) 
 
 
 
 
 
 0.0963 
 0.0387

### Carbon Reduction Plan Summary

In [8]:
# Increamental Cost
total_incremental_cost = sum(crm_year['incremental_cost'] for crm_year in data['building_analysis']['applied_carbon_reduction_measures'])

# Abatement Cost
total_marginal_abatement_cost = sum(crm_year['marginal_abatement_cost'] for crm_year in data['building_analysis']['applied_carbon_reduction_measures'])

# Emission Savings
total_carbon_emission_savings = sum(crm_year['annual_mean_carbon_emission_savings'] for crm_year in data['building_analysis']['applied_carbon_reduction_measures'])

df_metrics = pd.DataFrame(
    data=[total_incremental_cost, total_marginal_abatement_cost, total_carbon_emission_savings],
    index=['Total Incremental Cost','Average Abatement Cost per Tonne of Carbon Saved','Total Tonnes of Carbon Avoided'],
    columns=['']
).T

format_mapping={
    'Total Incremental Cost':'${:,.0f}' , 
    'Average Abatement Cost per Tonne of Carbon Saved':'${:,.2f}',
    'Total Tonnes of Carbon Avoided':'{:,.0f}'
}

# teui & carbon totals

# electricity
start_euie = sum(data['building_analysis']['applied_carbon_reduction_measures'][0]['surrogate_energy_model_output']['energy_use_intensity_electricity'].values())
# gas
start_euig = sum(data['building_analysis']['applied_carbon_reduction_measures'][0]['surrogate_energy_model_output']['energy_use_intensity_natural_gas'].values())
start_total_eui = int(start_euie + start_euig)
# start year
start_year = data['building_analysis']['applied_carbon_reduction_measures'][0]['year_applied']

# electricity
end_euie = sum(data['building_analysis']['applied_carbon_reduction_measures'][-1]['surrogate_energy_model_output']['energy_use_intensity_electricity'].values())
# gas
end_euig = sum(data['building_analysis']['applied_carbon_reduction_measures'][-1]['surrogate_energy_model_output']['energy_use_intensity_natural_gas'].values())
end_total_eui = int(end_euie + end_euig)
# end year
end_year = data['building_analysis']['applied_carbon_reduction_measures'][-1]['year_applied']

# ghgs
start_ghgs = int(data['building_analysis']['yearly_summaries'][0]['annual_carbon_emissions'])
end_ghgs = int(data['building_analysis']['yearly_summaries'][-1]['annual_carbon_emissions'])

# summary
summary_data = [
    [start_ghgs, end_ghgs, f'{int((start_ghgs-end_ghgs)/start_ghgs*100)}%'],
    [start_total_eui, end_total_eui, f'{int((start_total_eui-end_total_eui)/start_total_eui*100)}%']
]

summary_df = pd.DataFrame(
    data=summary_data,
    index=['Carbon Emissions (Tonnes CO₂e)','Energy Use Intensity (kWh/m²)'],
    columns=[f'{start_year}',f'{end_year}','Savings']
).T

#print dataframes side by side
space = "\xa0" * 15
df_metrics_styler = df_metrics.style.set_table_attributes("style='display:inline'")
summary_df_styler = summary_df.style.set_table_attributes("style='display:inline'")

display_html(df_metrics_styler.set_table_styles(df_style(header=16, body=16, max_width=150)).format(format_mapping)._repr_html_()
             +space+summary_df_styler.set_table_styles(df_style(header=16, body=16, max_width=150))._repr_html_()
             , raw=True)


Total Incremental Cost 
 Average Abatement Cost per Tonne of Carbon Saved 
 Total Tonnes of Carbon Avoided 
 
 
 
 
 
 $8,266 
 $77,358.70 
 822 
 
 
 
               
 
 
 
   
 Carbon Emissions (Tonnes CO₂e) 
 Energy Use Intensity (kWh/m²) 
 
 
 
 
 2023 
 901 
 132 
 
 
 2050 
 81 
 52 
 
 
 Savings 
 91% 
 60%

#### Yearly Carbon Reduction Measures

In [11]:
yearly_crms = pd.DataFrame.from_records(data['building_analysis']['applied_carbon_reduction_measures'])
cols = [
    'carbon_reduction_measure_type'
    , 'year_applied'
    , 'like_for_like_cost'
    , 'measure_cost'
    , 'incremental_cost'
    , 'simple_payback'
    , 'net_present_value'
    , 'internal_rate_of_return'
    , 'marginal_abatement_cost'
    , 'annual_mean_carbon_emission_savings'
    , 'annual_mean_utility_cost_savings'
]
format_mapping={
    'like_for_like_cost'                    : '${:,.0f}', 
    'measure_cost'                          : '${:,.0f}',
    'incremental_cost'                      : '${:,.0f}',
    'simple_payback'                        :  '{:,.1f}',
    'net_present_value'                     : '${:,.0f}',
    'internal_rate_of_return'               :  '{:,.1%}',
    'marginal_abatement_cost'               : '${:,.0f}',
    'annual_mean_carbon_emission_savings'   :  '{:,.0f}',
    'annual_mean_utility_cost_savings'      : '${:,.0f}'
}
yearly_crms_trim = yearly_crms.loc[:, cols]
yearly_crms_trim.replace(np.nan,0, inplace=True)

for key, value in format_mapping.items():
    yearly_crms_trim[key] = yearly_crms_trim[key].apply(value.format)


In [12]:
yearly_crms_trim.columns = yearly_crms_trim.columns.str.replace("_"," ")
yearly_crms_trim.columns = map(str.title, yearly_crms_trim.columns)
yearly_crms_trim.set_index('Carbon Reduction Measure Type',inplace=True)

In [13]:
yearly_crms_trim

,Year Applied,Like For Like Cost,Measure Cost,Incremental Cost,Simple Payback,Net Present Value,Internal Rate Of Return,Marginal Abatement Cost,Annual Mean Carbon Emission Savings,Annual Mean Utility Cost Savings
Carbon Reduction Measure Type,,,,,,,,,,
Base building state,2023,$0,$0,$0,0.0,$0,0.0%,$0,0,$0
Reduce Air Infiltration with Traditional Weatherization,2024,$0,$141,$141,140.6,$-138,-37.8%,"$3,122",4,$0
Install DCV Controls,2025,$0,"$1,050","$1,050",73.0,$-915,-24.7%,"$1,371",61,$14
No Action,2026,$0,$0,$0,0.0,$0,0.0%,$0,0,$0
Install Low-E Window Film with Climate Appropiate SHGC,2027,$0,"$2,206","$2,206","1,064.2","$-2,174",-23.5%,"$12,742",8,$2
Reduce Air Infiltration with Traditional Weatherization,2028,$0,$141,$141,140.6,$-160,0.0%,"$9,901",1,$-2
No Action,2029,$0,$0,$0,0.0,$0,0.0%,$0,0,$0
Install Domestic Hot Water Ambient Air-Source Heat Pump,2030,$56,$201,$145,1.3,"$1,242",166.4%,$-187,415,$109
Install Heat-Only Air Source Heat Pump,2031,$0,"$3,600","$3,600",40.0,"$-2,251",-6.0%,$342,329,$90
